In [1]:
import pandas as pd
import numpy as np
import jieba

In [2]:
from cleanse import separateUserContent,separateAllContent
from random import shuffle, randint
from gensim.models import word2vec
from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense ,SimpleRNN, Embedding, Activation, LSTM, GRU,Bidirectional,GlobalMaxPooling1D,Dot,Lambda,Add,Conv1D
from keras.models import Model,  Sequential, load_model,model_from_json
from keras import backend as K
from keras.callbacks import ModelCheckpoint

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
tmp_tup_df =pd.read_pickle("validation_tup_list.pkl")
validation_tup_list =tmp_tup_df.values.tolist()
print(validation_tup_list[0])

['我的T102H一直重開機不能用 平板電腦説找筆電處理IMG_0264.JPG IMG_0265.JPG像平板那款已跑10分鐘了 一直說處理更新', '目前可能是在更新 建議您先等他跑完']


In [4]:
validation_tup_list[33][1]

'HTTPS://WWW.ASUS.COM/TW/SUPPORT/FAQ/1002656/ 煩請您參考'

In [5]:
tmp_tup_df =pd.read_pickle("training_tup_list.pkl")
training_tup_list =tmp_tup_df.values.tolist()
print(training_tup_list[0])

['查型號什麽序號在那裡平版', '您可以再設定>關於>狀態>序號']


In [6]:
vec_model = word2vec.Word2Vec.load("word2vec.model")

In [7]:
import pickle #WORD INDEX 是WORD2VEC辭典的編號
f = open("word_index_dic.pkl","rb")
word_index = pickle.load(f)
f.close()

In [8]:
tmp_df =pd.read_pickle("Wrong_ans_int_set.pkl")#Wrong ANS SET紀錄每個問題對應的錯誤答案的編號
Wrong_ans_int_list =tmp_df.values
#print(tmp_Wrong_ans_int_list[100])
print(Wrong_ans_int_list[0])

[3816]


In [9]:
def makeQuesFeatuers(vec_model,word_index):
    docs = []

   
            
    for i in range(len(training_tup_list)):
        list1 = jieba.cut(training_tup_list[i][0], cut_all = False)
    
        docs.append(list(list1))

   
   

    encoded_docs = []
    for list10 in docs:
        list_temp = []
        for word in list10:
            if word in vec_model.wv.vocab:
                list_temp.append(word_index[word])
        encoded_docs.append(list_temp)

    #print(encoded_docs[0])
    max_length = 40
    padded_docs_list = pad_sequences(encoded_docs, maxlen = max_length, padding = 'post')

    padded_docs = np.array(padded_docs_list)

    return padded_docs
def makeAnsFeatuers(vec_model,word_index):
    docs = []

   
            
    for i in range(len(training_tup_list)):
        list1 = jieba.cut(training_tup_list[i][1], cut_all = False)
    
        docs.append(list(list1))

   
   

    encoded_docs = []
    for list10 in docs:
        list_temp = []
        for word in list10:
            if word in vec_model.wv.vocab:
                list_temp.append(word_index[word])
        encoded_docs.append(list_temp)

    #print(encoded_docs[0])
    max_length = 40
    padded_docs_list = pad_sequences(encoded_docs, maxlen = max_length, padding = 'post')

    padded_docs = np.array(padded_docs_list)

    return padded_docs
def makeWrongAnsFeatuers(vec_model,word_index,Wrong_ans_int_list):
    docs = []

    
            
    for i in range(len(training_tup_list)):#隨機找一句不是這句答案的答案
            
        #print(training_tup_list[i][1])
        #print(training_tup_list[tmp_int][1])
        tmp_int = Wrong_ans_int_list[i][0] #先找到編號在找句子
        list1 = jieba.cut(training_tup_list[tmp_int][1], cut_all = False)
    
        docs.append(list(list1))

   
   

    encoded_docs = []
    for list10 in docs:
        list_temp = []
        for word in list10:
            if word in vec_model.wv.vocab:
                list_temp.append(word_index[word])
        encoded_docs.append(list_temp)

    #print(encoded_docs[0])
    max_length = 40
    padded_docs_list = pad_sequences(encoded_docs, maxlen = max_length, padding = 'post')

    padded_docs = np.array(padded_docs_list)

    return padded_docs

In [10]:
train_q_fea = makeQuesFeatuers(vec_model, word_index)
train_a_fea = makeAnsFeatuers(vec_model,word_index)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.477 seconds.
Prefix dict has been built succesfully.


In [11]:
train_wa_fea = makeWrongAnsFeatuers(vec_model, word_index,Wrong_ans_int_list)

In [12]:
max_len = 40 #model的部份主要按照ANSWER SELECTION GITHUB 40是跟者上面的設定
input_q = Input((max_len,))
input_at = Input((max_len,))
input_aw= Input((max_len,))

In [13]:
embedding_matrix = np.zeros((len(vec_model.wv.vocab)+1, 250)) #250是WORD2VEC訓練的詞向量維度
for word, i in word_index.items():
        embedding_vector = vec_model.wv[word]
        embedding_matrix[i] = embedding_vector
        
embedding_layer = Embedding(len(vec_model.wv.vocab)+1, 250, weights = [embedding_matrix], input_length = 40)

In [14]:
emb_q = embedding_layer(input_q)
emb_at = embedding_layer(input_at)
emb_aw = embedding_layer(input_aw)

In [15]:
shared_lstm = Bidirectional(
    LSTM(300, return_sequences=True)#RETUREN SEQUENCE回傳所有時間點的隱藏曾的直
)
encode_q = shared_lstm(emb_q)
encode_at = shared_lstm(emb_at)
encode_aw = shared_lstm(emb_aw)

In [16]:
"""vec_q = GlobalMaxPooling1D()(encode_q)
m_a = Dense(1)(encode_at)
m_oq = Dense(40)(vec_q)
m_aq = Add()([m_a,m_oq])
m_aq = Activation("tanh")(m_aq)"""

'vec_q = GlobalMaxPooling1D()(encode_q)\nm_a = Dense(1)(encode_at)\nm_oq = Dense(40)(vec_q)\nm_aq = Add()([m_a,m_oq])\nm_aq = Activation("tanh")(m_aq)'

In [17]:
'''shared_conv = Conv1D(300,2,padding='same',activation='relu',strides=1)
conv_q = shared_conv(encode_q)
conv_at = shared_conv(encode_at)
conv_aw = shared_conv(encode_aw)'''

"shared_conv = Conv1D(300,2,padding='same',activation='relu',strides=1)\nconv_q = shared_conv(encode_q)\nconv_at = shared_conv(encode_at)\nconv_aw = shared_conv(encode_aw)"

In [18]:
vec_q = GlobalMaxPooling1D()(encode_q)
vec_at = GlobalMaxPooling1D()(encode_at)
vec_aw = GlobalMaxPooling1D()(encode_aw)

In [19]:
cosine_t = Dot(axes=1, normalize=True)([vec_q, vec_at])
cosine_w = Dot(axes=1, normalize=True)([vec_q, vec_aw])

In [20]:
neg = Lambda(lambda x: -x)
sub = Add()([cosine_w, neg(cosine_t)])#這邊開始的LOSS 主要按照論文李的HINGE LOSS 可以看一下我有沒有寫錯

In [21]:
def hinge(x):
    tmp = K.zeros(K.shape(x))
    return K.maximum(tmp, 0.1 + x) #錯誤的答案跟正確的至少可以相差0.2 再大就被視為一樣好 
    
hinge1 = Lambda(hinge)
result = hinge1(sub)

In [22]:
model = Model(inputs=[input_q, input_at, input_aw], outputs=[result, cosine_t, cosine_w])

In [23]:
model.compile(
    optimizer='adam', loss='mean_squared_error', loss_weights=[1, 0, 0]#只看HINGE LOSS 未來可以加入權重
)

In [24]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 40)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 40)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 40)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 40, 250)      901750      input_1[0][0]                    
                                                                 input_2[0][0]                    
          

In [25]:
label1 = np.zeros((len(train_a_fea), 1))#這邊地一個是HINGE LOSS 地2個是cos true  3 is cos  neg
label2 = np.ones((len(train_a_fea), 1))#cos 正確的最大是1
label3 = np.ones((len(train_a_fea), 1))
label3 = label3 * -1#COS錯誤的最好是-1
print(label3)

[[-1.]
 [-1.]
 [-1.]
 ...
 [-1.]
 [-1.]
 [-1.]]


In [28]:
filepath="weights_best_2.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',verbose=1, save_best_only=True,save_weights_only=False, mode='min')
callbacks_list = [checkpoint]

In [29]:
model.fit([train_q_fea,train_a_fea,train_wa_fea], [label1,label2,label3] , callbacks=callbacks_list,epochs= 10, batch_size = 256, verbose = 1)

Epoch 1/10
8394/8394 [==============================] - 11s 1ms/step - loss: 0.0033 - lambda_2_loss: 0.0033 - dot_1_loss: 0.0175 - dot_2_loss: 3.1860

Epoch 00001: loss improved from inf to 0.00335, saving model to weights_best_2.hdf5
Epoch 2/10
8394/8394 [==============================] - 9s 1ms/step - loss: 0.0017 - lambda_2_loss: 0.0017 - dot_1_loss: 0.0279 - dot_2_loss: 2.9102

Epoch 00002: loss improved from 0.00335 to 0.00173, saving model to weights_best_2.hdf5
Epoch 3/10
8394/8394 [==============================] - 9s 1ms/step - loss: 0.0011 - lambda_2_loss: 0.0011 - dot_1_loss: 0.0408 - dot_2_loss: 2.6895

Epoch 00003: loss improved from 0.00173 to 0.00105, saving model to weights_best_2.hdf5
Epoch 4/10
8394/8394 [==============================] - 9s 1ms/step - loss: 6.3936e-04 - lambda_2_loss: 6.3936e-04 - dot_1_loss: 0.0579 - dot_2_loss: 2.4783

Epoch 00004: loss improved from 0.00105 to 0.00064, saving model to weights_best_2.hdf5
Epoch 5/10
8394/8394 [=====================

In [30]:
model.load_weights(filepath ="weights_best_2.hdf5" )#先把最好的MODEL載入在除存
model_json = model.to_json()
with open("model_margin_0.1_epo_10_2.json", "w") as json_file:
    json_file.write(model_json)

In [31]:
test_ans_num = 99
tmp_df =pd.read_pickle("candidate_100_ans.pkl")#讀取對每個問題的100個答案
test_ans_set =tmp_df.values.tolist()

In [32]:
def makeTestQuesFeatuers(vec_model,word_index):
    docs = []

   
            
    for i in range(len(validation_tup_list)):
        list1 = jieba.cut(validation_tup_list[i][0], cut_all = False)
    
        docs.append(list(list1))

   
   

    encoded_docs = []
    for list10 in docs:
        list_temp = []
        for word in list10:
            if word in vec_model.wv.vocab:
                list_temp.append(word_index[word])
        encoded_docs.append(list_temp)

    #print(encoded_docs[0])
    max_length = 40
    padded_docs_list = pad_sequences(encoded_docs, maxlen = max_length, padding = 'post')

    padded_docs = np.array(padded_docs_list)

    return padded_docs

def makeTestAnsFeatures(vec_model,word_index, test_num):
    docs = []

   
            
    for i in range(len(test_ans_set)):
        list1 = jieba.cut(test_ans_set[i][test_num], cut_all = False)
    
        docs.append(list(list1))

   
   

    encoded_docs = []
    for list10 in docs:
        list_temp = []
        for word in list10:
            if word in vec_model.wv.vocab:
                list_temp.append(word_index[word])
        encoded_docs.append(list_temp)

    #print(encoded_docs[0])
    max_length = 40
    padded_docs_list = pad_sequences(encoded_docs, maxlen = max_length, padding = 'post')

    padded_docs = np.array(padded_docs_list)

    return padded_docs

In [33]:
test_q_fea = makeTestQuesFeatuers(vec_model, word_index)#產生出的矩陣是（ ,40)
test_a_fea = []
for i in range(test_ans_num+1): #有100個答案的FEATURE 產生出的矩陣是（100, , 40)有100組答案每一組答案是(,40)
    tmp_list = makeTestAnsFeatures(vec_model,word_index,i)
    test_a_fea.append(tmp_list)

In [34]:
ans = model.predict([test_q_fea,test_a_fea[0],test_a_fea[0]])#先預測第一句答案[RESULT , COS_POS COS_NEG因為要CONCAT
ans_cos_np = np.array(ans[1]).reshape((len(ans[1]),1))#從 (1,) to ( , 1)
#ans_neg_np = np.array(ans[2]).reshape((len(ans[1]),1))

In [35]:
for i in range(1,test_ans_num+1):#predict 剩下的 大約15分
    ans = model.predict([test_q_fea,test_a_fea[i],test_a_fea[i]])
    tmp_cos_np = np.array(ans[1]).reshape((len(ans[1]),1))
    ans_cos_np = np.concatenate((ans_cos_np,tmp_cos_np),axis = 1)#把答案串起來 ( ,100)
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [36]:
true_ans_np = ans_cos_np 

sorted_candidate_ans = []
for i in range(len(true_ans_np)):
    temp_list = []
    for j in range(0, test_ans_num+1):
        temp_list.append((test_ans_set[i][j],true_ans_np[i][j]))
    temp_list.sort(key=lambda tup: tup[1], reverse = True)
    sorted_candidate_ans.append(temp_list)

In [37]:
sum = 0
for i in range(len(sorted_candidate_ans)):
    if sorted_candidate_ans[i][0][1] == true_ans_np[i][99]:#最大的跟真實的答案一樣
        sum += 1
accuracy = sum / len(sorted_candidate_ans)
print(len(true_ans_np))
print(accuracy)

933
0.45444801714898175


In [38]:
sum = 0
for i in range(len(sorted_candidate_ans)):
    if sorted_candidate_ans[i][4][1] <= true_ans_np[i][99]:#真實的答案在第五明之前
        sum += 1
accuracy = sum / len(sorted_candidate_ans)
print(len(sorted_candidate_ans))
print(accuracy)

933
0.7438370846730975


In [39]:
with open("Output_test2.txt", "w") as text_file:
    for i in range(0, 100):
        text_file.write("Question %d : %s\n" % (i+1, validation_tup_list[i][0]))
        text_file.write("True answer %d : %s   %f \n" % (i+1, validation_tup_list[i][1], true_ans_np[i][99]))
        text_file.write("Ans:  \n\n")
        for j in range(0,5):
            text_file.write("%d. %s %f\n" % (j, sorted_candidate_ans[i][j][0], sorted_candidate_ans[i][j][1]))